In [52]:
%load_ext autoreload
%autoreload 2
import sys 
if '/Users/ericliu/Desktop/Latent-Dirichilet-Allocation' not in sys.path: 
    sys.path.append('/Users/ericliu/Desktop/Latent-Dirichilet-Allocation')

from collections import Counter, OrderedDict 

import torch as tr 
import numpy as np 

from src.utils import expec_log_dirichlet, expec_log_dirichlet_mirror, compute_elbo 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from scipy.special import psi, polygamma, gammaln
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
from nltk.corpus import reuters
from nltk.corpus import stopwords
import nltk 
nltk.download('reuters')

[nltk_data] Downloading package reuters to /Users/ericliu/nltk_data...


True

In [8]:
stops = stopwords.words("english")
stops += [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves", ".", "!", "?", ",", ";", ":", "[", "]", "{", "}", "-", "+", 
    "_", "/", "@", "#", "$", "%", "^", "&", "*", "(", ")", "<", ">", "|", "=",
    ".-", ".,", "'", '"', ',"'
]

In [6]:
def dg(gamma, d, i):
    """
    E[log θ_t] where θ_t ~ Dir(gamma)
    """
    return psi(gamma[d, i]) - psi(np.sum(gamma[d, :]))


def dl(lam, i, w_n):
    """
    E[log β_t] where β_t ~ Dir(lam)
    """
    return psi(lam[i, w_n]) - psi(np.sum(lam[i, :]))

In [12]:
reuters.words()[:10] 

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN']

In [13]:
# fetch titles only
# fetch 2000 docs only
trainset, testset = [], []
vocab = []

i = 0
for file_id in reuters.fileids():
    if file_id.startswith("train"):
        doc = [w.lower() for w in reuters.words(file_id) \
                 if (w.isupper()) \
                 if (w.lower() not in stops) \
                 and (not w.isnumeric())]
        if doc:
            trainset.append(doc)
            vocab += doc
            i += 1
    else:
        testset.append(
            [w.lower() for w in reuters.words(file_id) \
                         if (w.isupper()) \
                         and (w.lower() not in stops) \
                         and (not w.isnumeric())]
        )
    if i >= 2000:
        break

In [14]:
vocab = list(set(vocab))
word_to_ix = {w: i for i, w in enumerate(vocab)}

In [28]:
print(len(vocab))

4048


In [24]:
print(f"There are {len(trainset)} observations in trainset")
print(trainset[0])

print(f"There are {len(testset)} observations in testset")
print(testset[0])

There are 2000 observations in trainset
['bahia', 'cocoa', 'review', 'fob', 'u', 'u']
There are 3019 observations in testset
['asian', 'exporters', 'fear', 'damage', 'u', 'japan', 'rift', 'u', 'u', 'u', 'u', 'mc', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'u', 'miti']


In [29]:
for key in trainset[0]:
    print(word_to_ix[key])

1476
2151
2383
3925
1232
1232


In [15]:
def seq_to_ix(seq, vocab=vocab):
    # len(vocab), which is the last index, is for the <unk> (unknown) token
    unk_idx = len(vocab)
    return np.array(list(map(lambda w: word_to_ix.get(w, unk_idx), seq)))

data = {
    "train": list(map(seq_to_ix, trainset)),
    "test": list(map(seq_to_ix, testset))
}

In [19]:
data['train'][0]

array([1476, 2151, 2383, 3925, 1232, 1232])

In [78]:
class SmoothedLDA:
    
    def __init__(self, docs, vocab, k):
        self.docs = docs
        
        self.V = len(vocab)
        self.k = k  # number of topics
        self.N = np.array([doc.shape[0] for doc in docs])
        self.M = len(docs)
        
        V = self.V
        N = self.N
        M = self.M
        
        # initialize model parameters
        ##self.beta = np.ones((k, V)) / V
        self.alpha = np.random.gamma(100, 0.01, k)
        self.eta = np.ones(V)

        # initialize variational parameters
        # ϕ: (M x max(N) x k) arrays with zero paddings on the right
        self.phi = [np.ones((N[d], k)) / k for d in range(M)]
        self.gamma = self.alpha + (N / k).reshape(-1, 1)
        self.lam = np.random.gamma(shape=100, scale=0.01, size=(k, V))

    def vlb(self):
        """
        lower bound from variational inference
        """
        phi = self.phi
        gamma = self.gamma
        lam = self.lam
        alpha = self.alpha
        eta = self.eta
        docs = self.docs
        
        M = self.M
        k = self.k
        N = self.N
        V = self.V
        
        a0, a1, a2, a3_1, a3_2, a4, a5 = 0., 0., 0., 0., 0., 0., 0.

        a0 += (
            k * (
                gammaln(np.sum(eta)) 
                - np.sum(gammaln(eta))
            )
            + np.sum([(eta[j] - 1) * dl(lam, i, j) for j in range(V) for i in range(k)])
        )
        for d in range(M):

            a1 += (
                gammaln(np.sum(alpha))
                - np.sum(gammaln(alpha))
                + np.sum([(alpha[i] - 1) * dg(gamma, d, i) for i in range(k)])
            )

            a4 += (
                gammaln(np.sum(gamma[d, :]))
                - np.sum(gammaln(gamma[d, :]))
                + np.sum([(gamma[d, i] - 1) * dg(gamma, d, i) for i in range(k)])
            )
            
            for i in range(k):
                for j in range(V):
                    a3_2 += (
                        gammaln(np.sum(lam[i, j]))
                        - np.sum(gammaln(lam[i, :]))
                        + np.sum((lam[i, j] - 1) * dl(lam, i, j))
                    )

            for n in range(N[d]):
                w_n = int(docs[d][n])
                a2 += np.sum([phi[d][n, i] * dg(gamma, d, i) for i in range(k)])
                a3_1 += np.sum([phi[d][n, i] * dl(lam, i, w_n) for i in range(k)])
                a5 += np.sum([phi[d][n, i] * np.log(phi[d][n, i]) for i in range(k)])

        return a0 + a1 + a2 + a3_1 - a3_2 - a4 - a5

In [79]:
test_docs_str = trainset[0]
test_docs = [data['train'][0]]

print(test_docs_str)
print(test_docs)

['bahia', 'cocoa', 'review', 'fob', 'u', 'u']
[array([1476, 2151, 2383, 3925, 1232, 1232])]


In [80]:
mirror_model = SmoothedLDA(
    docs= test_docs,
    vocab = vocab, 
    k = 10, 
)

In [81]:
mirror_model.vlb()

1667503.6951637845

In [47]:
print(len(mirror_model.phi))
print(mirror_model.phi[0])

1
[[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]


In [59]:
word_cts = {}
word_pos = {}

for index, word in enumerate(test_docs_str): 

    
    word_cts[word]=word_cts.get(word, 0)

    if word not in word_pos: 

        word_pos[word] = []

    word_pos[word].append(index)

print(word_cts)
print(word_pos)


{'bahia': 0, 'cocoa': 0, 'review': 0, 'fob': 0, 'u': 0}
{'bahia': [0], 'cocoa': [1], 'review': [2], 'fob': [3], 'u': [4, 5]}


In [50]:
word_cts = Counter(test_docs_str)
word_inds = word_to_ix

print(word_cts)

Counter({'u': 2, 'bahia': 1, 'cocoa': 1, 'review': 1, 'fob': 1})


In [49]:
mirror_model.lam.shape

(10, 4048)

In [64]:
mirror_model.gamma.shape

(1, 10)

In [68]:
for doc in range(len(mirror_model.phi)):
    if not isinstance(mirror_model.phi[doc], tr.Tensor):
        mirror_model.phi[doc] = tr.tensor(mirror_model.phi[doc], dtype=float)
#mirror_model.phi 

In [77]:
compute_elbo(
    batch_size=len(test_docs), 
    _gamma_ = tr.tensor(mirror_model.gamma, dtype=float), 
    _phi_ = mirror_model.phi, 
    _lambda_ = tr.tensor(mirror_model.lam, dtype=float),
    _alpha_ = tr.tensor(mirror_model.alpha, dtype=float),
    _eta_ = tr.tensor(mirror_model.eta, dtype=float),
    word_cts = [word_cts],
    word_pos = [word_pos],
    word_to_inds = word_to_ix,  
)

tensor(-283.6431, dtype=torch.float64)